d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

In [0]:
# ANSWER
# Create widget for parameter passing into the notebook
dbutils.widgets.text("n_estimators", "100")
dbutils.widgets.text("learning_rate", ".1")
dbutils.widgets.text("max_depth", "1")

In [0]:
# ANSWER
# Read from the widget
n_estimators = int(dbutils.widgets.get("n_estimators"))        # Cast to int
learning_rate = float(dbutils.widgets.get("learning_rate"))    # Cast to float
max_depth = int(dbutils.widgets.get("max_depth"))              # Cast to int

In [0]:
# ANSWER
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

with mlflow.start_run() as run:
  # Import the data
  df = pd.read_csv("/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv")
  X_train, X_test, y_train, y_test = train_test_split(df.drop(["price"], axis=1), df[["price"]].values.ravel(), random_state=42)
    
  # Create model, train it, and create predictions
  gbr= GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth)
  gbr.fit(X_train, y_train)
  predictions = gbr.predict(X_test)

  # Log model
  model_path = "gbr-model"
  mlflow.sklearn.log_model(gbr, model_path)
    
  # Log params
  mlflow.log_param("n_estimators", n_estimators)
  mlflow.log_param("learning_rate", learning_rate)
  mlflow.log_param("max_depth", max_depth)

  # Log metrics
  mlflow.log_metric("mse", mean_squared_error(y_test, predictions))
  mlflow.log_metric("mae", mean_absolute_error(y_test, predictions))  
  mlflow.log_metric("r2", r2_score(y_test, predictions)) 
  
  artifactURI = mlflow.get_artifact_uri()
  model_output_path = artifactURI + "/" + model_path

In [0]:
# ANSWER
# Report the results back to the parent notebook
import json

dbutils.notebook.exit(json.dumps({
  "status": "OK",
  "model_output_path": model_output_path.replace("dbfs:", "/dbfs")
}))


-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>